## Homework

In this homework, we'll deploy the ride duration model in batch mode. Like in homework 1, we'll use the Yellow Taxi Trip Records dataset. 

You'll find the starter code in the [homework](homework) directory.

In [35]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [2]:
!python -V

Python 3.9.19


In [3]:
import pickle
import pandas as pd

In [4]:
with open("model.bin", "rb") as f_in:
    dv, model = pickle.load(f_in)

c:\Users\ramsi\anaconda3\envs\exp-tracking-env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ramsi\anaconda3\envs\exp-tracking-env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ["PULocationID", "DOLocationID"]



def read_data(filename):

    df = pd.read_parquet(filename)


    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

    df["duration"] = df.duration.dt.total_seconds() / 60


    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()


    df[categorical] = df[categorical].fillna(-1).astype("int").astype("str")

    return df

In [6]:
df = read_data(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet"
)

In [8]:
dicts = df[categorical].to_dict(orient="records")
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

## Q1. Notebook

We'll start with the same notebook we ended up with in homework 1.
We cleaned it a little bit and kept only the scoring part. You can find the initial notebook [here](homework/starter.ipynb).

Run this notebook for the March 2023 data.

What's the standard deviation of the predicted duration for this dataset?

* 1.24
* 6.24
* 12.28
* 18.28


In [9]:
y_pred.std()

6.247488852238703

## Q2. Preparing the output

Like in the course videos, we want to prepare the dataframe with the output. 

First, let's create an artificial `ride_id` column:

```python
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
```

Next, write the ride id and the predictions to a dataframe with results. 

Save it as parquet:

```python
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
```

What's the size of the output file?

* 36M
* 46M
* 56M
* 66M


In [10]:
df["predicted_duration"] = y_pred
year = 2023
month = 3
df["ride_id"] = f"{year:04d}/{month:02d}_" + df.index.astype("str")
df_result = df[["ride_id", "predicted_duration"]]
df_result.head()

,ride_id,predicted_duration
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486


In [11]:
output_file = "df_result.parquet"
df_result.to_parquet(
    output_file, engine="pyarrow", compression=None, index=False
)

In [16]:
%ls -lh -f '%z' "C:\\Users\\ramsi\\OneDrive\\Documents\\Github Projects\\mlops-zoomcamp\\04-deployment\\homework\\df_result.parquet"

The specified path is invalid.


In [18]:
%pwd

'c:\\Users\\ramsi\\OneDrive\\Documents\\Github Projects\\mlops-zoomcamp\\04-deployment\\homework'

In [24]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %code_wrap  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %mamba  %matplotlib  %micromamba  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%code_wrap  %%debug  %%file  %%html  %%javascript  %%js  %%latex 

In [23]:
%ls "df_result.parquet" -l --block-size=M

 Volume in drive C is Windows
 Volume Serial Number is 944C-4601

 Directory of c:\Users\ramsi\OneDrive\Documents\Github Projects\mlops-zoomcamp\04-deployment\homework


 Directory of c:\Users\ramsi\OneDrive\Documents\Github Projects\mlops-zoomcamp\04-deployment\homework


 Directory of c:\Users\ramsi\OneDrive\Documents\Github Projects\mlops-zoomcamp\04-deployment\homework


 Directory of c:\Users\ramsi\OneDrive\Documents\Github Projects\mlops-zoomcamp\04-deployment\homework

18-06-2024  00:56        68,641,880 df_result.parquet
               1 File(s)     68,641,880 bytes
               0 Dir(s)  522,794,328,064 bytes free


## Q3. Creating the scoring script

Now let's turn the notebook into a script. 

Which command you need to execute for that?

In [26]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 945 bytes to starter.py


## Q4. Virtual environment

Now let's put everything into a virtual environment. We'll use pipenv for that.

Install all the required libraries. Pay attention to the Scikit-Learn version: it should be the same as in the starter
notebook.

After installing the libraries, pipenv creates two files: `Pipfile`
and `Pipfile.lock`. The `Pipfile.lock` file keeps the hashes of the
dependencies we use for the virtual env.

What's the first hash for the Scikit-Learn dependency?

In [36]:
!conda list | grep scikit-learn

scikit-learn              1.4.2                    pypi_0    pypi


## Q5. Parametrize the script

Let's now make the script configurable via CLI. We'll create two 
parameters: year and month.

Run the script for April 2023. 

What's the mean predicted duration? 

* 7.29
* 14.29
* 21.29
* 28.29

Hint: just add a print statement to your script.

## Q6. Docker container 

Finally, we'll package the script in the docker container. 
For that, you'll need to use a base image that we prepared. 

This is what the content of this image is:
```
FROM python:3.10.13-slim

WORKDIR /app
COPY [ "model2.bin", "model.bin" ]
```

Note: you don't need to run it. We have already done it.

It is pushed it to [`agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim`](https://hub.docker.com/layers/agrigorev/zoomcamp-model/mlops-2024-3.10.13-slim/images/sha256-f54535b73a8c3ef91967d5588de57d4e251b22addcbbfb6e71304a91c1c7027f?context=repo),
which you need to use as your base image.

That is, your Dockerfile should start with:

```docker
FROM agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim

# do stuff here
```

This image already has a pickle file with a dictionary vectorizer
and a model. You will need to use them.

Important: don't copy the model to the docker image. You will need
to use the pickle file already in the image. 

Now run the script with docker. What's the mean predicted duration
for May 2023? 

* 0.19
* 7.24
* 14.24
* 21.19


docker build -t ride-duration-prediction-homework:v1 . docker run -it --rm ride-duration-prediction-homework:v1 2023 5

0.19